In [1]:
import numpy as np
import pandas as pd
import pickle
from nltk.tokenize import sent_tokenize,word_tokenize
from sklearn.metrics import accuracy_score
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('train.csv')
data.head()

,comment_text,id,identity_hate,insult,obscene,set,severe_toxic,threat,toxic,toxicity
0,explanation why the edits made under my userna...,0000997932d777bf,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
1,d aww he matches this background colour i m s...,000103f0d9cfb60f,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
2,hey man i m really not trying to edit war it...,000113f07ec002fd,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
3,more i can t make any real suggestions on im...,0001b41b1c6bb37e,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
4,you sir are my hero any chance you remember...,0001d958c54c6e35,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0


**A little description about the dataset**

Value of column identtify_hate is 1 if a comment is hate comment.   
Value of column insult is 1 if a comment is an insult.  
Same for sever_toxic,threat,toxic.

Also a comment can be classified into ***multiple classes*** at the same time, means a comment can be both hate and insult at the same time.  

Column toxicity contains the ***number of classes*** to which a comment belongs.

In [3]:
# Right now we are only interrested in finding if a comment is toxic or not so we extract only the comment col and toxicity col
x = data["comment_text"].values
y = data["toxicity"].values

In [4]:
data['toxicity'].value_counts()

0.0    143346
1.0      6360
3.0      4209
2.0      3480
4.0      1760
5.0       385
6.0        31
Name: toxicity, dtype: int64

We are only interested in binary classification of comment into toxic or non toxic and a comment is toxic if its y value is greater than or equal to 1. 

So lets merge all the comments whose y >= 1 into one single class.

In [5]:
y[y > 0] = 1
np.unique(y,return_counts="true")

(array([0., 1.]), array([143346,  16225], dtype=int64))

Out of 159,571 comments in the dataset 143346 belong to non toxic class and 16225 belong to toxic class.

Only 10% of the comments belong to toxic class. Clearly our dataset is **heavily imbalanced.**

This problem of imbalance dataset can be dealt in a better way but right now we decide to deal with this by **randomly selecting** 30000 comments from non toxic class and then merging them with toxic class.


In [6]:
# First filtering comments into non_toxic and toxic class.
non_toxic = x[y==0]
toxic = x[y==1]

In [7]:
# Randomly selecting 30000 non toxic comments.
ids = np.arange(non_toxic.shape[0])
np.random.shuffle(ids)
ids = ids[:30000]

non_toxic_selected = []

for i in ids:
    non_toxic_selected.append(non_toxic[i])

non_toxic_selected = np.asarray(non_toxic_selected)
print(non_toxic_selected.shape,toxic.shape)

(30000,) (16225,)


In [8]:
# Merging non toxic and toxic comments into one array
comments = np.concatenate((non_toxic_selected,toxic))
print(comments.shape)

(46225,)


Now we have an array 'comments' which contains 46225 commennts. Out of which first 30000 comments are non toxic and rest 16225 comments are toxic.

Percentage of toxic comments in our new dataset is 35%.

We're gonna have to define our new y array i.e. target array.

In [9]:
y_new = np.zeros((46225,),dtype="int")
y_new[30000:] = 1

In [10]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [11]:
class MEModelNB:

  # We're gonna use Multinomial Event Model Naive Bayes for text classification.
    
    def __init__(self):
        self.N = 0 # Total Documents
        self.Nc = [0,1] # List of all the classes
        self.count_Nc = [0,0] # contains the count of all the documents belonging to a particular class
        self.vocab = {}  # contains the vocab 
        self.V = 0 # total vocab size
        self.count_w_c = {} # contains frequency of word in a particular class
        self.count_c = [0,0] # contains total number of words occuring in the class by index
        self.prior_prob = [] # contains the prior probability of all the classes

    def commentPreProcessing(self,comment):
    
        """This function performs text preprocessing i.e. it removes stopwords and does 
    
        lemmatization and return all the useful words/tokens in the comment."""
     
        comment = comment.lower()
    
        # Performing Tokenization
        sentences = sent_tokenize(comment)
    
        words = []
    
        for sentence in sentences:
            c_words = word_tokenize(sentence)
        
            for w in c_words:
                if w not in words:
                    words.append(w)
    
        # Performing Lemmatization and stopword removal.
        words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]

        return words


    def train(self,comments,c_class):
    
        """ Accepts training data and trains the model. """
    
        m = len(comments)
        for i in range(m):
      
            comment = comments[i]
            c = c_class[i]

            # Step 1 - increment the count of total documents we have learned from N.
            self.N = self.N + 1

            # Step 2 - increment the count of documents that have been mapped to this category Nc.
            self.count_Nc[c] = self.count_Nc[c] + 1

            # Step 3 - Perform comment pre processing.
            words = self.commentPreProcessing(comment)

            # Step 4 - 
                #   4.1 if we encounter new words in this document, add them to our vocabulary,and update our vocabulary size |V|.
                #   4.2 update count(w, c ) => the frequency with which each word in the document has been mapped to this category.
                #   4.3 update count ( c ) => the total count of all words that have been mapped to this class.

            for w in words:
                if w in self.vocab:
                    self.vocab[w] = self.vocab[w] + 1
                    if(self.count_w_c[w][c] == 0):
                        self.count_c[c] += 1
          
                    self.count_w_c[w][c] += 1
                else:
                    self.vocab[w] = 1
                    self.count_w_c.setdefault(w,[0,0])
                    self.count_w_c[w][c] = 1
                    self.count_c[c] += 1
                    self.V = self.V + 1
    
        # Step 5 - Calculate prior probability of both the classes.
        for i in range(2): 
            prob = self.count_Nc[i]/float(self.N)
            self.prior_prob.append(prob)


    def predict(self,comment):
    
        """ This function accepts a comment and predicts whether it belongs to Toxic class or Non Toxic class."""

        # Step 1 - Peform comment pre processing.
        words = self.commentPreProcessing(comment)
    
        likelihood = [1,1]
    
        # We need to iterate through each word in the document 
        # and calculate: P(w|c) = [count(w,c) + 1]/[count(c) + |V|]

        # x = <w1,w2,w3....wn>, where x = comment whose class is to predicted
        # P(x|c) = Product(P(wi|c)) , c = class which is either toxic or non toxic.
    
        # We multiply each P(w|c) for each word w in the new document, then multiply by P(c) (Prior probability of class c)
        # and the result is the probability that this document belongs to this class.

        for c in self.Nc:
      
            for w in words:
                if w in self.vocab:
                    cond = (self.count_w_c[w][c] + 1)/(self.count_c[c] + self.V)
                else:
                    cond = 1/(self.count_c[c] + self.V)
          
                likelihood[c] = likelihood[c] * cond
            
        for i in range(2):
            likelihood[i] = likelihood[i] * self.prior_prob[i]

        # Predict the class which has highest P(Y=C|x) posterior probabiliy.
        pred = np.argmax(likelihood)
    
        if(pred == 1):
            return "Toxic"
        else:
            return "Non Toxic"

In [12]:
clf = MEModelNB()

In [13]:
clf.train(comments,y_new)

**Testing Time**

In [14]:
clf.predict('can you please shut up')

'Toxic'

In [15]:
clf.predict('shut up you shitty idiot')

'Toxic'

In [16]:
clf.predict('shut up you shitty idiot good')

'Toxic'

In [17]:
clf.predict('shut up you shitty idiot good nice thanks please')

'Toxic'

In [18]:
clf.predict('thanks a lot for explaining')

'Non Toxic'

In [19]:
clf.predict('nice explaination')

'Non Toxic'

In [20]:
clf.predict('Hi! how are you?')

'Non Toxic'

**Building Confusion Matrix and calculating F1 Score**

In [21]:
x_train, x_test, y_train, y_test = train_test_split(comments, y_new, test_size = 0.05)

In [22]:
classifier2 = MEModelNB()

In [23]:
classifier2.train(x_train,y_train)

In [24]:
y_pred = []
count = 0
for i in range(x_test.shape[0]):
    pred = classifier2.predict(x_test[i])
    if pred == "Toxic":
        y_pred.append(1)
    else:        
        y_pred.append(0)

In [25]:
y_pred = np.asarray(y_pred)
print(y_test.shape,y_pred.shape)

(2312,) (2312,)


In [26]:
confusion_matrix(y_test, y_pred)

array([[1476,   11],
       [ 423,  402]], dtype=int64)

In [27]:
f1_score(y_test,y_pred)

0.6494345718901454

**Future work**

Our model gives good prediction for sentences which contain toxic words but fails when it comes to sentences which do not contain toxic words but are still toxic.

Let's see an example.

In [28]:
clf.predict('not my fault you lack brain cells')

'Non Toxic'

In [29]:
clf.predict('not my fault you lack brain cells idiot')

'Toxic'

**Let's Save the model**

In [30]:
model = open('model.pkl', 'wb')

In [31]:
pickle.dump(clf,model)

In [32]:
model.close()